In [44]:
import numpy as np
import pandas as pd

In [45]:
samples = pd.read_csv('../titanic.csv')

In [46]:
samples

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500
...,...,...,...,...,...,...,...,...
882,0,2,Rev. Juozas Montvila,male,27.0,0,0,13.0000
883,1,1,Miss. Margaret Edith Graham,female,19.0,0,0,30.0000
884,0,3,Miss. Catherine Helen Johnston,female,7.0,1,2,23.4500
885,1,1,Mr. Karl Howell Behr,male,26.0,0,0,30.0000


In [47]:
def int_sex(row):
    if row['Sex'] == 'male':
        return 1
    else:
        return 0

samples['sex_int'] = samples.apply(lambda row: int_sex(row), axis=1)

In [48]:
from sklearn.model_selection import train_test_split

train_samples, test_samples = train_test_split(samples, test_size=0.2)

In [49]:
class Split:
    def __init__(self, attribute, cut_off, left_node, right_node):
        self.attribute = attribute
        self.cut_off = cut_off
        self.left_node = left_node
        self.right_node = right_node
        
    def predict(self, sample):
        if sample[self.attribute] < self.cut_off:
            return self.left_node.predict(sample)
        else:
            return self.right_node.predict(sample)
        
    def predict_proba(self, sample):
        if sample[self.attribute] < self.cut_off:
            return self.left_node.predict(sample)
        else:
            return self.right_node.predict(sample)        
        
    def __str__(self):
        return 'Split(' + str(self.attribute) + ', ' + str(self.cut_off) + \
            ', [' + str(self.left_node) + ', ' +  str(self.right_node) + '])'
        
class Leaf:
    def __init__(self, label):
        self.label = label

    def predict(self, sample):
        if self.label > 0.5:
            return 1
        else: 
            return 0
        
    def predict_proba(self, sample):
        return self.label                
        
    def __str__(self):
        return 'Leaf(' + str(self.label) + ')'
    

    

In [50]:
def H(a, b, a_plus_b):    
    #print('\t\t', a, b, a_plus_b)
    
    if a == 0 and b == 0:
        return 0.0
    
    p_a = float(a) / float(a_plus_b)
    p_b = float(b) / float(a_plus_b)
    
    log2_p_a = 0.0
    if a != 0:
        log2_p_a = np.log2(p_a)
        
    log2_p_b = 0.0
    if b != 0:
        log2_p_b = np.log2(p_b)        
        
    return -(p_a * log2_p_a + p_b * log2_p_b)

def split_score(num_plus_left, num_minus_left, num_plus_right, num_minus_right):
    
    #print('\tComputing score for ', num_plus_left, num_minus_left, num_plus_right, num_minus_right)
    
    num_left = num_plus_left + num_minus_left
    num_right = num_plus_right + num_minus_right
    num_plus = num_plus_left + num_plus_right
    num_minus = num_minus_left + num_minus_right

    num_samples = num_left + num_right

    # Prior "classification entropy" H_C(S)
    hcs = H(num_plus, num_minus, num_samples)

    # Entropy of S with respect to test T H_T(S)
    hts = H(num_left, num_right, num_samples)

    # Posterior "classification entropy" H_{C|T}(S) of S given the outcome of the test T
    p_sys = float(num_left) / float(num_samples)
    p_sns = float(num_right) / float(num_samples)

    hcsy = H(num_plus_left, num_minus_left, num_left)
    hcsn = H(num_plus_right, num_minus_right, num_right)

    hcts = p_sys * hcsy + p_sns * hcsn;

    # Information gain of applying test T
    icts = hcs - hcts;

    score = 2.0 * icts / (hcs + hts);
    
    return score

def split(samples, attribute_candidates, label_attribute, d):

    # TODO check that the attribute is non-constant
    
    attributes = np.random.choice(attribute_candidates, d, replace=False)
    
    max_score = -1
    the_attribute = None
    the_cutoff = None
    the_left_samples = None
    the_right_samples = None
    
    for attribute in attributes:
        attribute_values = np.array(samples[attribute])

        min_attribute_value = np.min(attribute_values)
        max_attribute_value = np.max(attribute_values)

        cut_off = np.random.uniform(min_attribute_value, max_attribute_value)    

        left_samples = samples[samples[attribute] < cut_off].copy(deep=True)
        right_samples = samples[samples[attribute] >= cut_off].copy(deep=True)

        num_plus_left = np.sum(left_samples[label_attribute] == 1)
        num_minus_left = np.sum(left_samples[label_attribute] == 0)
        num_plus_right = np.sum(right_samples[label_attribute] == 1)
        num_minus_right = np.sum(right_samples[label_attribute] == 0)

        score = split_score(num_plus_left, num_minus_left, num_plus_right, num_minus_right)
        
        if score > max_score:
            max_score = score
            the_attribute = attribute
            the_cutoff = cut_off
            the_left_samples = left_samples
            the_right_samples = right_samples            
    
    return the_attribute, the_cutoff, left_samples, right_samples, max_score    

def stop_split(samples, attribute_candidates, label_attribute, n_min):
    
    if len(samples) < n_min:
        return True
    
    for attribute in attribute_candidates:
        if len(samples[attribute].unique()) == 1:
            return True
    
    if len(samples[label_attribute].unique()) == 1:
        return True
    
    return False

def split_node(samples, attribute_candidates, label_attribute, d, n_min):
    
    if stop_split(samples, attribute_candidates, label_attribute, n_min):
        
        num_positive = float(np.sum(samples[label_attribute]))
        label = num_positive / len(samples[label_attribute])
        
        return Leaf(label)
    
    attribute, cut_off, left_samples, right_samples, score = \
        split(samples, attribute_candidates, label_attribute, d)
    
    #print('Splitting', len(samples), 'samples on', attribute, 'with cut_off', cut_off, 'and score', score, 
    #     len(left_samples), len(right_samples))
    
    left_child = split_node(left_samples, attribute_candidates, label_attribute, n_min, d)        
    right_child = split_node(right_samples, attribute_candidates, label_attribute, n_min, d)
    
    return Split(attribute, cut_off, left_child, right_child)
    

In [51]:
num_trees = 100
d = 5
n_min = 3

attribute_candidates = ['Age', 'Pclass', 'Fare', 'Siblings/Spouses Aboard', 'Parents/Children Aboard', 'sex_int']
label_attribute = 'Survived'
        
trees = []
for _ in range(0, num_trees):
    trees.append(split_node(train_samples, attribute_candidates, label_attribute, d, n_min))        

In [52]:
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

true_classes = test_samples[label_attribute]
predicted_classes = []

for row in range(0, len(test_samples)):
    
    predictions = [tree.predict(test_samples.iloc[row]) for tree in trees]
    num_positive = np.sum(predictions)
    if num_positive > len(predictions) / 2:
        predicted_class = 1
    else:
        predicted_class = 0
    
    predicted_classes.append(predicted_class)
    
accuracy_score(true_classes, predicted_classes)    

0.8146067415730337

In [53]:
f1_score(true_classes, predicted_classes)

0.7079646017699116

In [54]:
confusion_matrix(true_classes, predicted_classes)

array([[105,  15],
       [ 18,  40]])

In [57]:
from sklearn import tree

X_train = train_samples[attribute_candidates].values
y_train = train_samples[label_attribute].values

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

In [58]:
X_test = test_samples[attribute_candidates].values
y_test = test_samples[label_attribute].values

predictions = clf.predict(X_test)

accuracy_score(y_test, predictions)  

0.7471910112359551

In [59]:
f1_score(y_test, predictions)

0.64

In [60]:
confusion_matrix(y_test, predictions)

array([[93, 27],
       [18, 40]])